### Data
Downloaded from LMS

### Question 1.

Summary of data set: Each line of input file "heart.csv" represents a health record such as age, sex and chest pain type related to the heart disease. The column named "target" refers to the presence of heart disease in the patient. It is an integer valued 0 = disease, 1 = no disease.

Columns:
* age (numeric)
* sex (numeric)
* cp: chest pain type (4 integer values from 0 to 3)
* trestbps: resting blood pressure (numeric)
* chol: serum cholestoral in mg/dl (numeric)
* fbs: fasting blood sugar > 120 mg/dl (numeric)
* restecg: resting electrocardiographic results (integer values 0,1,2)
* thalach: maximum heart rate achieved (numeric)
* exang: exercise induced angina (1 = yes; 0 = no)
* oldpeak: ST depression induced by exercise relative to rest (numeric)
* slope: the slope of the peak exercise ST segment (valued 0, 1 or 2)
* ca: number of major vessels (0-3) colored by flourosopy (integer)
* thal: Thalium, a radioactive tracer injected during a stress test ranged from 0 to 3

Our goal is to learn a logistic regression model that predicts the presence of disease.

### 문제 1.

데이터셋 요약: 파일 "heart_disease_uci.csv"의 각 라인은 심장병과 관련된 환자의 건강지수를 나타낸다. 나이, 성별, 가슴통증 타입등과 같은 정보를 담고 있다. "target"이란 이름의 컬럼은 환자가 심장병이 있는 것으로 진단되었는지 여부를 나타내며 0이면 양성, 1이면 음성 판정이다.

문제: 주어진 데이터셋을 이용하여 로지스틱 회귀 모델을 학습하여 심장병 유무를 예측하라.

In [1]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

22/12/01 16:06:39 WARN Utils: Your hostname, orange resolves to a loopback address: 127.0.1.1; using 166.104.246.51 instead (on interface enp15s0)
22/12/01 16:06:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/01 16:06:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


(1) 우선 schema를 잘 정의하여 위 칼럼 설명에 나온 대로 float 타입과 integer 타입을 정의하여 데이터프레임 data을 생성하시오 (10점)

힌트: column 이름은 age, sex, dataset, cp, trestbps, chol, fbs, restecg, thalch, exang, oldpeak, 	slope, ca, thal, target

힌트: string type은 sex, dataset, cp, fbs, restecg, exang, slope, thal이고 integer type은 ca, target이며 나머지는 float type으로 정의하시오.

In [22]:
from pyspark.sql.types import *

schema = StructType([
    StructField('age', FloatType(), True),
    StructField('sex', StringType(), True),
    StructField('dataset', StringType(), True),
    StructField('cp', StringType(), True),
    StructField('trestbps', FloatType(), True),
    StructField('chol', FloatType(), True),
    StructField('fbs', StringType(), True),
    StructField('restecg', StringType(), True),
    StructField('thalch', FloatType(), True),
    StructField('exang', StringType(), True),
    StructField('oldpeak', FloatType(), True),
    StructField('slope', StringType(), True),
    StructField('ca', IntegerType(), True),
    StructField('thal', StringType(), True),
    StructField('target', IntegerType(), True),
])

data = spark.read.format('csv')\
                 .option('header', 'true')\
                 .option("quote", "\"")\
                 .option("escape", "\"")\
                 .option("multiLine", "true")\
                 .schema(schema)\
                 .load('../../data/heart_disease_uci.csv')

data.show(5)
data.count()

+----+------+---------+---------------+--------+-----+-----+--------------+------+-----+-------+-----------+---+------------+------+
| age|   sex|  dataset|             cp|trestbps| chol|  fbs|       restecg|thalch|exang|oldpeak|      slope| ca|        thal|target|
+----+------+---------+---------------+--------+-----+-----+--------------+------+-----+-------+-----------+---+------------+------+
|63.0|  Male|Cleveland| typical angina|   145.0|233.0| TRUE|lv hypertrophy| 150.0|FALSE|    2.3|downsloping|  0|fixed defect|     0|
|37.0|  Male|Cleveland|    non-anginal|   130.0|250.0|FALSE|        normal| 187.0|FALSE|    3.5|downsloping|  0|      normal|     0|
|41.0|Female|Cleveland|atypical angina|   130.0|204.0|FALSE|lv hypertrophy| 172.0|FALSE|    1.4|  upsloping|  0|      normal|     0|
|56.0|  Male|Cleveland|atypical angina|   120.0|236.0|FALSE|        normal| 178.0|FALSE|    0.8|  upsloping|  0|      normal|     0|
|57.0|Female|Cleveland|   asymptomatic|   120.0|354.0|FALSE|        n

676

In [23]:
data.printSchema()

root
 |-- age: float (nullable = true)
 |-- sex: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- cp: string (nullable = true)
 |-- trestbps: float (nullable = true)
 |-- chol: float (nullable = true)
 |-- fbs: string (nullable = true)
 |-- restecg: string (nullable = true)
 |-- thalch: float (nullable = true)
 |-- exang: string (nullable = true)
 |-- oldpeak: float (nullable = true)
 |-- slope: string (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: string (nullable = true)
 |-- target: integer (nullable = true)



(1-1) 우선 Null을 하나라도 포함한 row를 모두 제거하시오.

In [24]:
data = data.dropna()
data.count()

216

(2) integer type의 컬럼들은 값이 category 이기 때문에 logistic regression을 사용할때 이를 다차원 실수벡터로 변환하여 많이 분석한다. 예를 들면 color 라는 컬럼이 있고 0:red, 1:blue, 3:black이라면 0값을 갖는다고 color가 낮은 것이 아니고 3값을 갖는다고 color가 높은 것이 아니다. 이런 경우 3차원 벡터로 확장하여 red인 경우 (0, 0, 1), blue인경우 (0, 1, 0), black인경우 (1, 0, 0)과 같이 실수벡터로 확장, 치환하여 분석을 많이 한다. 이러한 방법을 one hot encoding이라고 한다. 

string type의 컬럼들을 모두 이와 같이 one hot encoding 하여 새로운 데이터프레임을 생성해 data에 덮어쓰시오. one hot encoding에 앞서서 string indexing을 해야 함을 잊지 않도록. (15점)

In [25]:
from pyspark.ml.feature import StringIndexer
string_cols = ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
string_indexer = StringIndexer(inputCols=string_cols, outputCols=[i + '_index' for i in string_cols])
data = string_indexer.fit(data).transform(data)
data.select(*[i + '_index' for i in string_cols]).show(5)

+---------+-------------+--------+---------+-------------+-----------+-----------+----------+
|sex_index|dataset_index|cp_index|fbs_index|restecg_index|exang_index|slope_index|thal_index|
+---------+-------------+--------+---------+-------------+-----------+-----------+----------+
|      0.0|          0.0|     3.0|      1.0|          1.0|        0.0|        2.0|       2.0|
|      0.0|          0.0|     1.0|      0.0|          0.0|        0.0|        2.0|       0.0|
|      1.0|          0.0|     2.0|      0.0|          1.0|        0.0|        0.0|       0.0|
|      0.0|          0.0|     2.0|      0.0|          0.0|        0.0|        0.0|       0.0|
|      1.0|          0.0|     0.0|      0.0|          0.0|        1.0|        0.0|       0.0|
+---------+-------------+--------+---------+-------------+-----------+-----------+----------+
only showing top 5 rows



In [26]:
from pyspark.ml.feature import OneHotEncoder
categorical_cols = ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal', 'ca']
onehotencoder = OneHotEncoder(inputCols=[i + '_index' for i in string_cols] + ['ca'], outputCols=[i + '_ohe' for i in categorical_cols])
data = onehotencoder.fit(data).transform(data)
data.select(*[i + '_ohe' for i in categorical_cols]).show(5)

+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|      sex_ohe|  dataset_ohe|       cp_ohe|      fbs_ohe|  restecg_ohe|    exang_ohe|    slope_ohe|     thal_ohe|       ca_ohe|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|(1,[0],[1.0])|(2,[0],[1.0])|    (3,[],[])|    (1,[],[])|(2,[1],[1.0])|(1,[0],[1.0])|    (2,[],[])|    (2,[],[])|(3,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|(3,[1],[1.0])|(1,[0],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|    (2,[],[])|(2,[0],[1.0])|(3,[0],[1.0])|
|    (1,[],[])|(2,[0],[1.0])|(3,[2],[1.0])|(1,[0],[1.0])|(2,[1],[1.0])|(1,[0],[1.0])|(2,[0],[1.0])|(2,[0],[1.0])|(3,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|(3,[2],[1.0])|(1,[0],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|(2,[0],[1.0])|(2,[0],[1.0])|(3,[0],[1.0])|
|    (1,[],[])|(2,[0],[1.0])|(3,[0],[1.0])|(1,[0],[1.0])|(2,[0],[1.0])|    (1,[],[])|(2,[0],[1.0])|(2,[0

(3) 다음은 분석에 사용할 컬럼을 feature vector로 변환하여 dataframe에 추가하기 위해, 우선 분석에 사용할 컬럼 이름의 리스트를 뽑아내는 일이다. 다음 셀에 분석에 사용할 컬럼 이름의 리스트를 cols에 저장하는 코드를 작성하시오. 위 StringIndexer와 OneHotEncoder에서 입력으로 사용했던 컬럼들은 제거하고. (10점)

In [29]:
cols = data.columns
cols.remove('target')
for i in categorical_cols:
    cols.remove(i)
    if i != 'ca':
        cols.remove(i+'_index')
cols

['age',
 'trestbps',
 'chol',
 'thalch',
 'oldpeak',
 'sex_ohe',
 'dataset_ohe',
 'cp_ohe',
 'fbs_ohe',
 'restecg_ohe',
 'exang_ohe',
 'slope_ohe',
 'thal_ohe',
 'ca_ohe']

(4) 이제 VectorAssembler를 이용해서 cols의 컬럼들을 한데 합쳐 feature vector를 만들어 data에 다시 넣도록 하자. 새로 추가될 feature vector 의 컬럼의 이름은 "features"라고 하자. (10점)

In [33]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(inputCols = cols, outputCol='features')
data = vector_assembler.transform(data)
data.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|(22,[0,1,2,3,4,5,...|
|(22,[0,1,2,3,4,5,...|
|(22,[0,1,2,3,4,6,...|
|[56.0,120.0,236.0...|
|(22,[0,1,2,3,4,6,...|
+--------------------+
only showing top 5 rows



(5) 데이터프레임 data를 8대2의 비율로 나누어 각각 train, test란 이름의 데이터프레임으로 쪼갠다. pyspark.sql.DataFrame.randomSplit 함수를 사용하고 seed는 편의상 1을 주기로 하자. (10점)

In [34]:
train, test = data.randomSplit([0.8, 0.2], 1)
train.count(), test.count()

(175, 41)

(6) 이제 LogisticRegression을 이용하여 학습을 시작한다. 학습에는 데이터프레임 train의 칼럼 features를 사용한다. 그리고 LogisticRegression 모델의 학습된 결과는 model 이란 이름의 변수명을 사용하여 저장한다. 또한 모델 생성자 생성 시, 예측결과는 default대로 prediction 컬럼에 쓰도록 설정하고 iteration은 10회만 실시하도록 설정한다. LogisticRegression 모델의 생성자와 모델이 다름을 주의할 것. (10점)

In [48]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='target', maxIter=10)
model = lr.fit(train)

(7) model을 이용해 데이터프레임 test의 각 row에 대해 심장병 유무를 예측한다. 예측결과가 추가된 데이터프레임은 predict_test란 이름으로 저장한다. (10점)

In [50]:
predict_test = model.transform(test)

(8) predict_test의 결과에 대해 정확도와 거짓양성(false positive)와 거짓음성(false negative)의 비율을 계산한다. (25점)

* 정확도는 전체 데이터 중 예측결과 prediction와 참 값 target이 같은 비율을 말한다.
* 거짓양성은 예측결과가 양성 (즉, 병이 있음. 다시 말해 prediction = 0)인 데이터 중 틀린 비율 (즉, 병이 없음; target = 1)을 말한다.
* 거짓음성은 예측결과가 음성 (즉, 병이 없음; prediction = 1)인 데이터 중 틀린 비율 (즉, 병이 있음; target = 0)을 말한다.

In [54]:
real_pos_cnt = predict_test.rdd.map(lambda row: 1 if row['target'] == 1 else 0).reduce(lambda x, y: x + y)
pred_pos_cnt = predict_test.rdd.map(lambda row: 1 if row['prediction'] == 1 else 0).reduce(lambda x, y: x + y)
TP = predict_test.rdd.map(lambda row: 1 if row['target'] == 1 and row['target'] == row['prediction'] else 0).reduce(lambda x, y: x + y)
correct_cnt = predict_test.rdd.map(lambda row: 1 if row['target'] == row['prediction'] else 0).reduce(lambda x, y: x + y)
print(f'accuracy : {correct_cnt / predict_test.count()}')
print(f'precision : {TP/pred_pos_cnt}')
print(f'recall : {TP/real_pos_cnt}')

accuracy : 0.7317073170731707
precision : 0.5454545454545454
recall : 0.5


In [55]:
sc.stop()